In [18]:
from langchain_core.documents import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain.llms import Ollama
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from typing import List
from langchain_core.runnables import RunnableLambda 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

## Vector stores and retrievers


In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
llm = ChatOllama(model="llama3.2:latest")

In [4]:
#embeddings = (OllamaEmbeddings(model="nomic-embed-text:latest")
#)
embeddings = (OllamaEmbeddings(model="llama3.2:latest")
)

In [5]:
vectorstore = Chroma.from_documents(documents, embeddings)

In [6]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.9999994039535522),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  0.9999994039535522),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.5967835187911987),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.604480266571045)]

In [13]:
await vectorstore.asimilarity_search_with_score("cat")

[(Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.9999994039535522),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  0.9999994039535522),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.5967835187911987),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.604480266571045)]

In [16]:
ret = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

In [17]:
ret.batch(["cat", "dog"])

[[Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [21]:
message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages("human", message)

rag_chain={"context":ret, "question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("Tell me about dogs.")
print(response)

ValueError: Unsupported template format: 
Answer this question using the provided context only.
{question}

Context:
{context}
